In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
data = pandas.read_csv('C:\\Users\\hackadmin\\Desktop\\DvnHAck2019\\ESPLiftFlat.txt', sep=",")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#METRICS
data.columns

In [3]:
data_length = len(data)
list_of_good_columns = []

for each_column in data.columns:
    print(each_column, data[each_column].dtype)

    if data[each_column].dtype == 'object':
        data[each_column] = data[each_column].replace({'NULL': np.nan})

print(" ")

Id int64
ESP object
TimeStamp object
Amperage float64
AtmosphericTemperature float64
CasingPressure float64
DownholeIntakePressure float64
DownholeIntakeTemp float64
Frequency float64
HeaterTreaterRollup float64
HeaterTreaterPressure float64
MotorTemp float64
ProdGasMeterRollup float64
ProdGasMeterFlowRate float64
SeparatorRollup float64
SeparatorPressure float64
StatusControlMode object
TubingPressure float64
Voltage float64
PIIntTSTicks int64
PIIntShapeID int64
 


In [4]:
#METRICS
for each_column in data.columns:
    data_available = data[data[each_column].notnull()]
    if (len(data_available)/float(data_length)) < 0.1:
        list_of_good_columns.append(each_column)
        print (each_column, len(data_available)/float(data_length))

HeaterTreaterRollup 0.0
ProdGasMeterRollup 0.0
SeparatorRollup 0.0
SeparatorPressure 0.042298356931882816


In [5]:
# remove unused data columns
del data['HeaterTreaterRollup']
del data['ProdGasMeterRollup']
del data['SeparatorRollup']
del data['SeparatorPressure']
del data['PIIntTSTicks']
del data['PIIntShapeID']
del data['AtmosphericTemperature']

In [ ]:
#METRICS
for each_column in data.columns:
    print(each_column)

In [ ]:
#METRICS
for each_column in data.columns:
    data_available = data[data[each_column].notnull()]
    if (len(data_available)/float(data_length)) > 0.1:
        list_of_good_columns.append(each_column)
        print (each_column, len(data_available)/float(data_length))

In [ ]:
#METRICS
data.describe()

In [7]:
# this creates a dictionary of data frames
data_frame_list = {}
for each_data in data['ESP'].unique():
    data_frame_list[each_data] = data[data.ESP == each_data]
    data_frame_list[each_data] = data_frame_list[each_data].ffill(axis = 0)
    data_frame_list[each_data] = data_frame_list[each_data].bfill(axis = 0)

In [8]:
#resampling to 10 min data
for key in data_frame_list:
    data_frame_list[key]['TimeStamp'] = pandas.to_datetime(data_frame_list[key]['TimeStamp']) 
    data_frame_list[key] = data_frame_list[key].resample('720min', on = 'TimeStamp').mean()
    data_frame_list[key].head()
    #data.plot(y='Amperage')

In [9]:
for each_data in data['ESP'].unique():
    data_frame_list[each_data]['Anamoly1'] = 0
    data_frame_list[each_data].loc[((data_frame_list[each_data]['Amperage'] < 50) | (data_frame_list[each_data]['Frequency'] < 30) | (data_frame_list[each_data]['Voltage'] < 50)),'Anamoly1'] = 1
    
    
    data_frame_list[each_data]['Anamoly2'] = 0
    data_frame_list[each_data].loc[((data_frame_list[each_data]['Amperage'] > 250) | (data_frame_list[each_data]['DownholeIntakeTemp'] > 350) | (data_frame_list[each_data]['Frequency'] > 70) | (data_frame_list[each_data]['MotorTemp'] > 350) | (data_frame_list[each_data]['Voltage'] > 500)),'Anamoly2'] = 1
    
    data_frame_list[each_data]['Anamoly3a'] = data_frame_list[each_data]['Anamoly1'].shift(-1)
    data_frame_list[each_data]['Anamoly3b'] = data_frame_list[each_data]['Anamoly1'].shift(-2)
    data_frame_list[each_data][['Anamoly3a', 'Anamoly3b']] = data_frame_list[each_data][['Anamoly3a', 'Anamoly3b']].fillna(value=0)
    data_frame_list[each_data]['Anamoly3'] = (data_frame_list[each_data]['Anamoly3a'] + data_frame_list[each_data]['Anamoly3b'])/(data_frame_list[each_data]['Anamoly3a'] + data_frame_list[each_data]['Anamoly3b'])

In [10]:
data_frame_list.pop('MLT FED 19-303668-1XTPH ESP 1', None)
data_frame_list.pop('SDU BONER BROS FED 22-153771-1XPH ESP 1', None)

,Id,Amperage,CasingPressure,DownholeIntakePressure,DownholeIntakeTemp,Frequency,HeaterTreaterPressure,MotorTemp,ProdGasMeterFlowRate,StatusControlMode,TubingPressure,Voltage,Anamoly1,Anamoly2,Anamoly3a,Anamoly3b,Anamoly3
TimeStamp,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2799248.5,0.0,71.677300,6554.0,5554.0,0.0,45.832100,5554.0,206.767487,NaN,235.424805,NaN,1,1,1.0,1.0,1.0
2019-01-01 12:00:00,2799968.5,0.0,71.747231,6554.0,5554.0,0.0,45.832100,5554.0,206.767487,NaN,234.930295,NaN,1,1,1.0,1.0,1.0
2019-01-02 00:00:00,2800688.5,0.0,71.817162,6554.0,5554.0,0.0,45.832100,5554.0,206.767487,NaN,234.435786,NaN,1,1,1.0,1.0,1.0
2019-01-02 12:00:00,2801408.5,0.0,71.887092,6554.0,5554.0,0.0,45.832100,5554.0,206.767487,NaN,233.941277,NaN,1,1,1.0,1.0,1.0
2019-01-03 00:00:00,2802128.5,0.0,71.957023,6554.0,5554.0,0.0,45.832100,5554.0,206.767487,NaN,233.446768,NaN,1,1,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-30 00:00:00,3146228.5,0.0,48.792558,6554.0,5554.0,0.0,37.958820,5554.0,212.525615,NaN,306.450106,NaN,1,1,1.0,1.0,1.0
2019-08-30 12:00:00,3146948.5,0.0,47.477465,6554.0,5554.0,0.0,38.981950,5554.0,195.403006,NaN,306.770541,NaN,1,1,1.0,1.0,1.0
2019-08-31 00:00:00,3147668.5,0.0,46.801396,6554.0,5554.0,0.0,36.377073,5554.0,196.591823,NaN,307.090975,NaN,1,1,1.0,1.0,1.0


In [11]:
for key in data_frame_list:
    data_frame_list[key]['ESP'] = key

In [12]:
output_df = pandas.DataFrame()
for key in data_frame_list:
    output_df = output_df.append(data_frame_list[key])

output_df.head()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Amperage,Anamoly1,Anamoly2,Anamoly3,Anamoly3a,Anamoly3b,CasingPressure,DownholeIntakePressure,DownholeIntakeTemp,ESP,Frequency,HeaterTreaterPressure,Id,MotorTemp,ProdGasMeterFlowRate,StatusControlMode,TubingPressure,Voltage
TimeStamp,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,118.0,0,0,NaN,0.0,0.0,73.010005,587.0,153.0,CU DOWNS FED 15-223769-2XTPH ESP 1,64.0,52.968792,360.5,181.0,725.716675,NaN,422.879720,472.0
2019-01-01 12:00:00,118.0,0,0,NaN,0.0,0.0,79.422686,587.0,153.0,CU DOWNS FED 15-223769-2XTPH ESP 1,64.0,52.968792,1080.5,181.0,725.716675,NaN,460.022479,472.0
2019-01-02 00:00:00,118.0,0,0,NaN,0.0,0.0,85.835368,587.0,153.0,CU DOWNS FED 15-223769-2XTPH ESP 1,64.0,52.968792,1800.5,181.0,725.716675,NaN,497.165239,472.0
2019-01-02 12:00:00,118.0,0,0,NaN,0.0,0.0,92.248050,587.0,153.0,CU DOWNS FED 15-223769-2XTPH ESP 1,64.0,52.968792,2520.5,181.0,725.716675,NaN,534.307998,472.0
2019-01-03 00:00:00,118.0,0,0,NaN,0.0,0.0,85.295918,587.0,153.0,CU DOWNS FED 15-223769-2XTPH ESP 1,64.0,52.968792,3240.5,181.0,725.716675,NaN,494.040792,472.0


In [ ]:
output_df.to_csv('C:\\Users\\hackadmin\\Desktop\\DvnHAck2019\\ESP_clean_resample_with_anomolies_12hrs.csv')

In [ ]:
output_df.head()

In [ ]:
output_df.columns

# Model

In [58]:
x_list =[] 
y_list =[]
    
for index, rows in output_df.iterrows(): 
    my_list =[rows.DownholeIntakeTemp, rows.MotorTemp] 
    
    x_list.append(my_list)
    y_list.append(rows.Anamoly3)

print(y_list) 

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1.0, 1.0, 1.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
output_df2 = output_df
output_df2 = output_df2.dropna(how='all')
output_df2 = output_df2.fillna(0)

print(output_df2['DownholeIntakeTemp'].max())
print(output_df2['MotorTemp'].max())

print(output_df2['DownholeIntakeTemp'].min())
print(output_df2['MotorTemp'].min())

X = output_df2[['DownholeIntakeTemp', 'MotorTemp']]
y = output_df2['Anamoly3']

225.30416666666667
3608.940772708329
0.0
0.0


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X.values.tolist(), y.values.tolist(), test_size = 0.20)

In [62]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [63]:
y_pred = svclassifier.predict(X_test)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[679  43]
 [ 97  58]]
              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91       722
         1.0       0.57      0.37      0.45       155

    accuracy                           0.84       877
   macro avg       0.72      0.66      0.68       877
weighted avg       0.82      0.84      0.83       877



In [43]:
import pickle
filename = 'model_2_2days.pkl'
pickle.dump(svclassifier, open("C:\\Users\\hackadmin\\Desktop\\DvnHAck2019\\"+filename, 'wb'))